In [1]:
import numpy as np
import pandas as pd
import pathlib
import sys
from datetime import date,datetime,timedelta

import xarray as xr

ROOT_PATH = pathlib.Path().resolve().parents[0]
sys.path.append(ROOT_PATH)

In [90]:
basedate = datetime(2022,1,1)

In [3]:
xrdf = xr.open_dataset(ROOT_PATH/basedate.strftime("data/%Y/%m/%Y_%m%d.nc"))
amemaster = pd.read_csv(ROOT_PATH/"ame_master_20230323.csv")

In [4]:
amemaster_kan = amemaster[(amemaster["種類"] == "官")].drop_duplicates(subset="観測所番号")

In [55]:
amemaster_kan.iloc[7,:]

都府県振興局                         留萌
観測所番号                       13277
種類                              官
観測所名                           留萌
ｶﾀｶﾅ名                         ﾙﾓｲ
気象情報等に表記する名称                留萌市大町
所在地             留萌市大町　留萌特別地域気象観測所
緯度(度)                          43
緯度(分)                        56.7
経度(度)                         141
経度(分)                        37.9
海面上の高さ(ｍ)                      24
風速計の高さ(ｍ)                    16.3
温度計の高さ(ｍ)                       －
観測開始年月日                  昭55.6.21
備考1                         13915
備考2                             －
Name: 51, dtype: object

In [11]:
lat1 = 43
lat2 = 56.7
lon1 = 141
lon2 = 37.9

lat = lat1 + lat2 / 60
lon = lon1 + lon2 / 60
print(lat,lon)

45.415 141.67833333333334


In [35]:
int(45.415 / 0.04999977) * 0.04999977

45.39979116

In [5]:
for _,i in amemaster_kan.iterrows():
    print(i["緯度(度)"],i["経度(度)"])

45 141
45 141
45 141
44 142
43 142
43 142
44 141
43 141
43 141
42 141
43 141
43 141
42 140
42 140
44 142
44 143
44 143
44 144
43 144
43 144
43 145
43 144
42 144
42 143
42 143
42 143
42 141
42 140
42 142
41 140
41 140
42 139
41 140
41 141
40 140
40 140
40 139
40 141
40 140
39 140
39 140
39 141
39 141
39 141
39 141
38 141
38 140
38 140
38 139
38 139
38 140
38 140
38 140
37 140
37 139
37 140
37 140
36 140
36 140
36 140
36 139
36 139
36 139
36 139
35 139
35 139
35 139
34 139
34 139
34 139
34 139
34 139
34 139
33 139
33 139
27 142
24 153
35 138
35 140
35 140
35 140
35 140
34 139
35 139
36 138
36 138
36 137
36 137
36 138
35 137
35 138
35 138
35 138
35 139
34 138
34 137
34 138
34 138
34 138
35 136
34 136
34 137
36 137
35 136
34 136
34 136
34 136
34 136
38 138
37 139
37 139
37 138
36 137
36 137
36 137
37 136
37 136
36 136
36 136
36 136
35 136
35 136
35 135
35 135
34 135
34 135
34 135
34 135
35 134
34 134
34 135
34 135
34 134
34 135
34 135
33 135
33 135
35 134
34 133
34 133
34 132
34 133
34 132

In [ ]:
amemaster_kan

In [14]:
x = xrdf.to_dataframe()

In [15]:
x_df = x.rename_axis(['lon', 'lat', 'time', 'ref_time']).reset_index()

In [ ]:
x_df["lat"].unique()

In [56]:
lat1 = 43
lat2 = 56.7
lon1 = 141
lon2 = 37.9

lat = lat1 + lat2 / 60
lon = lon1 + lon2 / 60

print(lat,lon)
x_df[(round(x_df["lon"],2) == round(int(lon / 0.0625) * 0.0625, 2))
    &
    (round(x_df["lat"],2) == round(int(lat / 0.04999977) * 0.04999977,2))
    &
    (x_df["ref_time"] == "2022-01-01 00:00:00")
    &
    (x_df["time"] == "2022-01-01 00:00:00")
    ]

43.945 141.63166666666666


,lon,lat,time,ref_time,psea,sp,u,v,temp,rh,r1h,ncld_upper,ncld_mid,ncld_low,ncld,dswrf
33562368,141.625,43.900002,2022-01-01,2022-01-01,100901.834754,100026.605137,-3.938838,-0.550459,264.017178,79.174312,200.0,0.000002,18.310001,18.066668,27.246668,665.0


In [70]:
pd.DataFrame().columns.shape[0] == 0

array([], dtype=bool)

In [96]:
df = pd.DataFrame()
for _,i in amemaster_kan.iterrows():
    lon = i["経度(度)"] + i["経度(分)"] / 60
    lat = i["緯度(度)"] + i["緯度(分)"] / 60
    # print(f"\r{lon},{lat}",end="")
    tmpdf = x_df[(round(x_df["lon"],2) == round(int(lon / 0.0625) * 0.0625, 2))
                &
                (round(x_df["lat"],2) == round(int(lat / 0.04999977) * 0.04999977,2))
                &
                (pd.to_datetime(x_df["ref_time"]).dt.hour == 0)
                &
                (pd.to_datetime(x_df["time"]) == basedate)
                ]
    # display(tmpdf)
    df = pd.concat([df.reset_index(drop=True),tmpdf.reset_index(drop=True)],axis=0)
    break

In [97]:
df

,lon,lat,time,ref_time,psea,sp,u,v,temp,rh,r1h,ncld_upper,ncld_mid,ncld_low,ncld,dswrf
0,141.625,45.400002,2022-01-01,2022-01-01,100777.064114,100564.219806,4.483181,-1.828746,266.75089,55.862386,200.0,0.000002,58.691666,29.491668,58.691666,665.0
